In [ ]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm


import os
import pickle

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('Data/X_test.csv', header=None, delimiter=',').values

Y_train_21 = pd.read_csv('Data/Y_train_std_21.csv', header=None, delimiter=',').values
Y_test_21 = pd.read_csv('Data/Y_test_std_21.csv', header=None, delimiter=',').values

Y_train_std = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',').values
Y_test_std = pd.read_csv('Data/Y_test_std.csv', header=None, delimiter=',').values


train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_21 = torch.tensor(Y_train_21, dtype=torch.float32)
test_y_21 = torch.tensor(Y_test_21, dtype=torch.float32)

train_y = torch.tensor(Y_train_std, dtype=torch.float32)
test_y = torch.tensor(Y_test_std, dtype=torch.float32)


torch.set_default_dtype(torch.float32)

####################################################################

Device = 'cpu'


output_file = 'Result/L.DKMGP_21_result.csv'
mcmc_dir = 'Result/L.DKMGP_21_mcmc_result'
if not os.path.exists(mcmc_dir):
    os.makedirs(mcmc_dir)

if not os.path.exists(output_file):
    with open(output_file, 'w') as f:
        f.write('Iteration,test_preds,estimated_params\n')

row_idx = 0

In [ ]:

input_point = test_y_21[row_idx, :]

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_21, k=500)

MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_NNMultitaskGP(
    local_train_x, local_train_y, n_tasks = local_train_y.shape[1], 
    feature_extractor_class = FeatureE.FeatureExtractor_4, covar_type = 'RQ', 
    lr=0.05, num_iterations=5000, patience=10, device = Device)

preds_tmp = Prediction.preds_for_one_model(
    MultitaskGP_models, MultitaskGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)
    ).cpu().detach().numpy()



In [ ]:
bounds = bound.get_bounds(local_train_x)

estimated_params_tmp, _ = Estimation.multi_start_estimation(
    MultitaskGP_models, MultitaskGP_likelihoods, row_idx, test_y_21, bounds,
    Estimation.estimate_params_for_one_model_Adam, num_starts=4, num_iterations=1000, lr=0.01,
    patience=10, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
)

In [ ]:
def preds_distribution(model, likelihood, xxx):
    model.eval()
    likelihood.eval()
    preds = likelihood(model(xxx))
    return preds

In [ ]:
def preds_distribution_fast_pred_var(model, likelihood, xxx):
    model.eval()
    likelihood.eval()
    with gpytorch.settings.fast_pred_var():
        preds = likelihood(model(xxx))
    return preds

In [ ]:
estimated_params_tmp

In [ ]:
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az


def run_mcmc_Uniform_initial_params(
    Pre_function, Models, Likelihoods, row_idx, test_y, bounds,
    num_sampling=2000, warmup_step=1000, num_chains=1, device='cpu',
    initial_params=None
):
    test_y = test_y.to(dtype=torch.float32, device=device)

    bounds = [
        (
            torch.tensor(b[0], dtype=torch.float32, device=device),
            torch.tensor(b[1], dtype=torch.float32, device=device)
        ) for b in bounds
    ]

    def model():
        params = []
        for i, (min_val, max_val) in enumerate(bounds):
            param_i = pyro.sample(f'param_{i}', dist.Uniform(min_val, max_val))
            params.append(param_i)

        theta = torch.stack(params)
        gp_pred = Pre_function(Models, Likelihoods, theta.unsqueeze(0))
        y_obs = test_y[row_idx, :]
        pyro.sample('obs', gp_pred, obs=y_obs)

    nuts_kernel = NUTS(model)

    initial_params_dict = {
        f'param_{i}': torch.tensor(p, dtype=torch.float32, device=device)
        for i, p in enumerate(initial_params)
    }

    mcmc = MCMC(nuts_kernel, num_samples=num_sampling, warmup_steps=warmup_step, num_chains=num_chains, initial_params=initial_params_dict)
    mcmc.run()



    return mcmc



In [ ]:
mcmc_result_Uniform_fast_pred_var = run_mcmc_Uniform_initial_params(
    preds_distribution_fast_pred_var, MultitaskGP_models, MultitaskGP_likelihoods, 
    row_idx, test_y_21, bounds, 
    num_sampling=2000, warmup_step=1000, num_chains=1, device=Device, initial_params=estimated_params_tmp
)
posterior_samples_Uniform_fast_pred_var = mcmc_result_Uniform_fast_pred_var.get_samples()


In [ ]:
mcmc_result_Uniform_fast_pred_var = Estimation.run_mcmc_Uniform(
    preds_distribution_fast_pred_var, MultitaskGP_models, MultitaskGP_likelihoods, 
    row_idx, test_y_21, bounds, 
    num_sampling=1200, warmup_step=300, num_chains=1, device=Device
)
posterior_samples_Uniform_fast_pred_var = mcmc_result_Uniform_fast_pred_var.get_samples()


In [ ]:
mcmc_result_Uniform = Estimation.run_mcmc_Uniform(
    preds_distribution, MultitaskGP_models, MultitaskGP_likelihoods, 
    row_idx, test_y_21, bounds, 
    num_sampling=1200, warmup_step=300, num_chains=1, device=Device
)
posterior_samples_Uniform_nograd = mcmc_result_Uniform.get_samples()


In [ ]:
loaded_samples_chain1 = torch.load("Result/L.MGP_21_mcmc_result/result_2.pt", map_location=Device)